In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract data science jobs csv into DataFrame

In [2]:
# Read in the data science jobs csv
datasci_file = "../Resources/datascientist_jobs_in_australia_Oct_25_2019.csv"
datasci_df = pd.read_csv(datasci_file)

In [3]:
# Summary of DataFrame
datasci_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619 entries, 0 to 618
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  619 non-null    int64  
 1   title       619 non-null    object 
 2   company     619 non-null    object 
 3   cpage       619 non-null    object 
 4   ratings     619 non-null    float64
 5   location    619 non-null    object 
 6   days_ago    619 non-null    int64  
 7   summary     619 non-null    object 
dtypes: float64(1), int64(2), object(5)
memory usage: 38.8+ KB


### Transform data science jobs csv

In [4]:
# Rename the unnamed column to id and set as index
datasci_df.rename(columns={"Unnamed: 0":"id"},inplace=True)
datasci_df.set_index("id",inplace=True)

In [5]:
# Preview resulting DataFrame
datasci_df.head()

,title,company,cpage,ratings,location,days_ago,summary
id,,,,,,,
1,Deloitte Access Economics - Junior Data Scientist,Deloitte,https://au.indeed.com/cmp/Deloitte,4.0,Sydney NSW,30,The data will be both temporal and spatial. Ad...
2,Data Scientist,Australian Government Department of Human Serv...,https://au.indeed.com/cmp/Australian-Governmen...,3.9,Canberra ACT,10,They lead the implementation of data science a...
3,Junior Data Analyst/Scientist,International Institute of Data & Analytics,https://au.indeed.comn/a,0.0,Sydney NSW,10,"In data science and big data analytics, the ID..."
4,IBM Research Scientist Data Scientist – Melbourne,IBM,https://au.indeed.com/cmp/IBM,3.9,Melbourne VIC,30,"Experience working with big data sets, especia..."
5,Data Scientist,Cash Converters,https://au.indeed.com/cmp/Cash-Converters,3.5,Taringa QLD,1,Collaborates with data/software engineers to i...


In [6]:
# Check for duplicate rows, based on all columns
duplicate_rows = datasci_df[datasci_df.duplicated(keep="first")]
# duplicate_rows
# duplicate_rows.count()

In [7]:
# Drop duplicate rows from DataFrame
datasci_df.drop_duplicates(inplace=True)

In [8]:
# Clean up cpage column (replace invalid URL 'https://au.indeed.comn/a' with 'Not available')
datasci_df.replace("https://au.indeed.comn/a","Not available",inplace=True)

In [9]:
# Replace 0.0 ratings with 'No rating'
datasci_df.replace(0.00,"No rating",inplace=True)

In [10]:
# Drop days_ago columns
datasci_df.drop(columns="days_ago",inplace=True)

In [11]:
datasci_df

,title,company,cpage,ratings,location,summary
id,,,,,,
1,Deloitte Access Economics - Junior Data Scientist,Deloitte,https://au.indeed.com/cmp/Deloitte,4,Sydney NSW,The data will be both temporal and spatial. Ad...
2,Data Scientist,Australian Government Department of Human Serv...,https://au.indeed.com/cmp/Australian-Governmen...,3.9,Canberra ACT,They lead the implementation of data science a...
3,Junior Data Analyst/Scientist,International Institute of Data & Analytics,Not available,No rating,Sydney NSW,"In data science and big data analytics, the ID..."
4,IBM Research Scientist Data Scientist – Melbourne,IBM,https://au.indeed.com/cmp/IBM,3.9,Melbourne VIC,"Experience working with big data sets, especia..."
5,Data Scientist,Cash Converters,https://au.indeed.com/cmp/Cash-Converters,3.5,Taringa QLD,Collaborates with data/software engineers to i...
...,...,...,...,...,...,...
564,Senior Research Analyst,Hudson,https://au.indeed.com/cmp/Hudson,3.6,Melbourne VIC,This role will lead to candidates who have a s...
578,6M Contract - Automation Engineer - JavaScript...,Real Time Australia,Not available,No rating,Melbourne VIC,You will be working with an exceptional cross ...
579,Full Stack Senior Software Engineer (React.js ...,Real Time Australia,Not available,No rating,Melbourne VIC,Working on building a unique data driven platf...


In [12]:
# Clean up state only values  in location
location_cols = ["location"]
location_df = datasci_df[location_cols].copy()
# Find state only values in location
location_missing_df = location_df.loc[location_df["location"].isin(["New South Wales","Victoria","Queensland","Tasmania"])]
location_missing_df

,location
id,
7,Victoria
28,Victoria
133,New South Wales
208,New South Wales
312,Queensland
427,Tasmania


In [13]:
# Based in index values, replace state only values in location with "Not avilable <state>" to enable splitting of location into two columns
datasci_df["location"].loc[[7,28,133,208,312,427]] = ["Not available VIC","Not available VIC","Not available NSW","Not available NSW","Not available QLD","Not available TAS"]

In [14]:
datasci_df.loc[[7,28,133,208,312,427]]

,title,company,cpage,ratings,location,summary
id,,,,,,
7,Data Scientist,UniSuper,https://au.indeed.com/cmp/Unisuper,3.9,Not available VIC,"Strong TSQL skills for data discovery, data pa..."
28,Data Scientist - Marketing,World Vision Australia,https://au.indeed.com/cmp/World-Vision-Australia,No rating,Not available VIC,Developing data assets (mainly using SQL and R...
133,Data Scientist - Digital Information Services,Talent International,https://au.indeed.com/cmp/Talent-International,No rating,Not available NSW,Degree in Information Technology or related fi...
208,Research Analyst,Dexus,https://au.indeed.com/cmp/Dexus,4.2,Not available NSW,Work on one of the best property portfolios in...
312,Senior Pricing Analyst,Auto & General Insurance,https://au.indeed.com/cmp/Auto-&-General-Insur...,4.1,Not available QLD,As part of Pricing you will work closely with ...
427,Machine Learning Engineer,Betsson Group,Not available,No rating,Not available TAS,"Understanding of data structures, data modelin..."


In [15]:
# Separate city and state into two columns
location = datasci_df["location"]
location
# location.unique()
# # location.value_counts()
location_new_df = location.str.rsplit(" ",n=1,expand=True)
location_new_df

,0,1
id,,
1,Sydney,NSW
2,Canberra,ACT
3,Sydney,NSW
4,Melbourne,VIC
5,Taringa,QLD
...,...,...
564,Melbourne,VIC
578,Melbourne,VIC
579,Melbourne,VIC


In [16]:
# Add new location and state columns into existing DataFrame
datasci_df["city"] = location_new_df[0]
datasci_df["state"] = location_new_df[1]
# Drop existing location column
datasci_df.drop(columns="location",inplace=True)

In [17]:
# Check city and state data
# datasci_df["city"].unique()
# datasci_df["state"].unique()

In [18]:
# Rename columns with meaningful titles e.g. 'Indeed landing page'
datasci_df.rename(columns={"title": "job posting title",
                            "company": "organisation",
                            "cpage": "Indeed profile page",
                            "ratings": "Indeed rating",
                            "summary": "job summary",
                            "city": "location"},
                            inplace=True)

In [19]:
# Select columns from data sci DF to create a job information table
jobinfo_cols = ["job posting title","organisation","location","state"]
jobinfo_df = datasci_df[jobinfo_cols].copy()

In [20]:
# Sort data by organisation and job posting title; rest index
jobinfo_df = jobinfo_df.sort_values(by=["organisation","job posting title"], ascending=True)
jobinfo_df.reset_index(drop=True,inplace=True)

In [21]:
jobinfo_df

,job posting title,organisation,location,state
0,Full Stack (Nodejs/Reactjs) Developer,2XM Technology Pty Ltd,Sydney,NSW
1,Junior Data Analyst,2hats,Sydney,NSW
2,Market Research Analyst - Australia ANVa,A V Immigration And Careers Consultancy Privat...,Australia,None
3,Senior Sleep Scientist,ACT Government,Canberra,ACT
4,Senior Sleep Scientist,ACT Government,Canberra,ACT
...,...,...,...,...
491,iOS Software Engineer,hipages,Sydney,NSW
492,Data Analyst| People Analytics | HR Data Analy...,inDmand,Sydney,NSW
493,Data Scientist | People Analytics | HR Data An...,inDmand,Sydney,NSW
494,Guest (Customer) Insights Analyst,kikki.K,Australia,None


In [22]:
# Select columns to create organisation DF
orginfo_cols = ["organisation","Indeed profile page","Indeed rating"]
orginfo_df = datasci_df[orginfo_cols].copy()
orginfo_df

,organisation,Indeed profile page,Indeed rating
id,,,
1,Deloitte,https://au.indeed.com/cmp/Deloitte,4
2,Australian Government Department of Human Serv...,https://au.indeed.com/cmp/Australian-Governmen...,3.9
3,International Institute of Data & Analytics,Not available,No rating
4,IBM,https://au.indeed.com/cmp/IBM,3.9
5,Cash Converters,https://au.indeed.com/cmp/Cash-Converters,3.5
...,...,...,...
564,Hudson,https://au.indeed.com/cmp/Hudson,3.6
578,Real Time Australia,Not available,No rating
579,Real Time Australia,Not available,No rating


In [23]:
# Drop duplicates from org DF
orginfo_df.drop_duplicates(subset="organisation",inplace=True)

In [24]:
# Sort data by organisation
orginfo_df = orginfo_df.sort_values(by=["organisation"], ascending=True)
orginfo_df.reset_index(drop=True,inplace=True)
orginfo_df

,organisation,Indeed profile page,Indeed rating
0,2XM Technology Pty Ltd,Not available,No rating
1,2hats,Not available,No rating
2,A V Immigration And Careers Consultancy Privat...,Not available,No rating
3,ACT Government,https://au.indeed.com/cmp/Act-Government,3.3
4,ACT Health,https://au.indeed.com/cmp/Act-Health,3
...,...,...,...
260,h2o.ai,Not available,No rating
261,hipages,Not available,No rating
262,inDmand,Not available,No rating
263,kikki.K,https://au.indeed.com/cmp/Kikki.k,2.6


### Extract ABS labour market data csv into DataFrame



In [25]:
# Read in csv
abslabour_file = "../Resources/SA4 Time Series - July 2020.csv"
abslabour_df = pd.read_csv(abslabour_file)
abslabour_df.head()

,State/Territory,Date,Employment Rate (15-64),Unemployment Rate (15+),Participation Rate (15+)
0,NSW,Feb-78,NaN,6.9,60.8
1,NSW,Mar-78,NaN,6.4,59.8
2,NSW,Apr-78,63.7,6.5,60.2
3,NSW,May-78,63.7,6.1,60.0
4,NSW,Jun-78,63.7,6.4,60.1


### Transform ABS labour market data

In [26]:
# Filter for relevant labour market data dates (Oct 2019 matched jobs data)
abslabour_oct2019_df = abslabour_df.loc[abslabour_df["Date"] == "Oct-19"]

In [27]:
# Drop row with value "Australia" in state/territory column
abslabour_oct2019_df.drop(4580,inplace=True)

In [28]:
# Rename columns
abslabour_oct2019_df.columns=["state","date","employment rate", "unemployment rate", "participation rate"]

In [29]:
# Sort data by state
abslabour_oct2019_df = abslabour_oct2019_df.sort_values(by=["state"], ascending=True)
abslabour_oct2019_df.reset_index(drop=True,inplace=True)

In [30]:
# Select columns from abslabour DF to create a labour market information table
labourinfo_cols = ["state","employment rate","unemployment rate","participation rate"]
labourinfo_df = abslabour_oct2019_df[labourinfo_cols].copy()

In [31]:
# Set index to state
labourinfo_df.set_index("state",inplace=True)
labourinfo_df

,employment rate,unemployment rate,participation rate
state,,,
ACT,79.1,3.0,70.8
NSW,74.9,4.8,65.5
NT,74.3,5.6,73.4
QLD,73.9,6.4,66.0
SA,72.0,6.2,62.7
TAS,71.2,6.0,60.6
VIC,74.0,4.8,66.4
WA,74.7,5.7,68.2


### List of final DFs for database tables

In [32]:
# Job information
jobinfo_df.index.name = "id"
jobinfo_df

,job posting title,organisation,location,state
id,,,,
0,Full Stack (Nodejs/Reactjs) Developer,2XM Technology Pty Ltd,Sydney,NSW
1,Junior Data Analyst,2hats,Sydney,NSW
2,Market Research Analyst - Australia ANVa,A V Immigration And Careers Consultancy Privat...,Australia,None
3,Senior Sleep Scientist,ACT Government,Canberra,ACT
4,Senior Sleep Scientist,ACT Government,Canberra,ACT
...,...,...,...,...
491,iOS Software Engineer,hipages,Sydney,NSW
492,Data Analyst| People Analytics | HR Data Analy...,inDmand,Sydney,NSW
493,Data Scientist | People Analytics | HR Data An...,inDmand,Sydney,NSW


In [33]:
# Organisation information
orginfo_df.index.name = "id"
orginfo_df

,organisation,Indeed profile page,Indeed rating
id,,,
0,2XM Technology Pty Ltd,Not available,No rating
1,2hats,Not available,No rating
2,A V Immigration And Careers Consultancy Privat...,Not available,No rating
3,ACT Government,https://au.indeed.com/cmp/Act-Government,3.3
4,ACT Health,https://au.indeed.com/cmp/Act-Health,3
...,...,...,...
260,h2o.ai,Not available,No rating
261,hipages,Not available,No rating
262,inDmand,Not available,No rating


In [34]:
# Labour market information
labourinfo_df

,employment rate,unemployment rate,participation rate
state,,,
ACT,79.1,3.0,70.8
NSW,74.9,4.8,65.5
NT,74.3,5.6,73.4
QLD,73.9,6.4,66.0
SA,72.0,6.2,62.7
TAS,71.2,6.0,60.6
VIC,74.0,4.8,66.4
WA,74.7,5.7,68.2


### Create database connection

In [35]:
connection_string = "postgres:postgres1985@localhost:5432/datascijobs_db"
engine = create_engine(f'postgresql://{connection_string}')

In [36]:
# Confirm tables
engine.table_names()

[]

### Load DataFrames into database

In [37]:
labourinfo_df.to_sql(name='labour_market_information', con=engine, if_exists='append', index=True)

In [38]:
orginfo_df.to_sql(name='organisation_information', con=engine, if_exists='append', index=True)

In [39]:
jobinfo_df.to_sql(name='job_information', con=engine, if_exists='append', index=True)